# 双人龙与地下城

在这个笔记本中，我们展示如何使用 [CAMEL](https://www.camel-ai.org/) 的概念来模拟一个包含主角和地下城主的角色扮演游戏。为了模拟这个游戏，我们创建一个 `DialogueSimulator` 类来协调两个代理之间的对话。

## 导入 LangChain 相关模块

In [1]:
from typing import Callable, List

from langchain.schema import (
    HumanMessage,
    SystemMessage,
)
from langchain_openai import ChatOpenAI

## `DialogueAgent` 类
`DialogueAgent` 类是 `ChatOpenAI` 模型的一个简单封装，它通过简单地将消息作为字符串连接来存储 `dialogue_agent` 视角的消息历史。

它提供了两个方法：
- `send()`：对消息历史应用聊天模型并返回消息字符串
- `receive(name, message)`：将由 `name` 说出的 `message` 添加到消息历史中

In [ ]:
class DialogueAgent:
    def __init__(
        self,
        name: str,
        system_message: SystemMessage,
        model: ChatOpenAI,
    ) -> None:
        self.name = name
        self.system_message = system_message
        self.model = model
        self.prefix = f"{self.name}: "
        self.reset()

    def reset(self):
        self.message_history = ["下面是到目前为止的对话。"]

    def send(self) -> str:
        """
        对消息历史应用聊天模型并返回消息字符串
        """
        message = self.model.invoke(
            [
                self.system_message,
                HumanMessage(content="\n".join(self.message_history + [self.prefix])),
            ]
        )
        return message.content

    def receive(self, name: str, message: str) -> None:
        """
        将由 {name} 说出的 {message} 添加到消息历史中
        """
        self.message_history.append(f"{name}: {message}")

## `DialogueSimulator` 类
`DialogueSimulator` 类接受一个代理列表。在每一步中，它执行以下操作：
1. 选择下一个发言者
2. 调用下一个发言者发送消息
3. 将消息广播给所有其他代理
4. 更新步骤计数器。
下一个发言者的选择可以实现为任何函数，但在这种情况下，我们只是简单地循环遍历代理。

In [ ]:
class DialogueSimulator:
    def __init__(
        self,
        agents: List[DialogueAgent],
        selection_function: Callable[[int, List[DialogueAgent]], int],
    ) -> None:
        self.agents = agents
        self._step = 0
        self.select_next_speaker = selection_function

    def reset(self):
        for agent in self.agents:
            agent.reset()

    def inject(self, name: str, message: str):
        """
        用 {name} 的 {message} 开始对话
        """
        for agent in self.agents:
            agent.receive(name, message)

        # 增加时间
        self._step += 1

    def step(self) -> tuple[str, str]:
        # 1. 选择下一个发言者
        speaker_idx = self.select_next_speaker(self._step, self.agents)
        speaker = self.agents[speaker_idx]

        # 2. 下一个发言者发送消息
        message = speaker.send()

        # 3. 每个人接收消息
        for receiver in self.agents:
            receiver.receive(speaker.name, message)

        # 4. 增加时间
        self._step += 1

        return speaker.name, message

## 定义角色和任务

In [ ]:
protagonist_name = "Harry Potter"
storyteller_name = "Dungeon Master"
quest = "Find all of Lord Voldemort's seven horcruxes."
word_limit = 50  # 任务头脑风暴的字数限制

## 让 LLM 为游戏描述添加细节

In [ ]:
game_description = f"""这是一个龙与地下城游戏的主题：{quest}。
        游戏中有一个玩家：主角 {protagonist_name}。
        故事由讲述者 {storyteller_name} 讲述。"""

player_descriptor_system_message = SystemMessage(
    content="你可以为龙与地下城玩家的描述添加细节。"
)

protagonist_specifier_prompt = [
    player_descriptor_system_message,
    HumanMessage(
        content=f"""{game_description}
        请用不超过 {word_limit} 个字的创意描述来描述主角 {protagonist_name}。
        直接对 {protagonist_name} 说话。
        不要添加任何其他内容。"""
    ),
]
protagonist_description = ChatOpenAI(temperature=1.0)(
    protagonist_specifier_prompt
).content

storyteller_specifier_prompt = [
    player_descriptor_system_message,
    HumanMessage(
        content=f"""{game_description}
        请用不超过 {word_limit} 个字的创意描述来描述讲述者 {storyteller_name}。
        直接对 {storyteller_name} 说话。
        不要添加任何其他内容。"""
    ),
]
storyteller_description = ChatOpenAI(temperature=1.0)(
    storyteller_specifier_prompt
).content

In [ ]:
print("主角描述：")
print(protagonist_description)
print("讲述者描述：")
print(storyteller_description)

Protagonist Description:
"Harry Potter, you are the chosen one, with a lightning scar on your forehead. Your bravery and loyalty inspire all those around you. You have faced Voldemort before, and now it's time to complete your mission and destroy each of his horcruxes. Are you ready?"
Storyteller Description:
Dear Dungeon Master, you are the master of mysteries, the weaver of worlds, the architect of adventure, and the gatekeeper to the realm of imagination. Your voice carries us to distant lands, and your commands guide us through trials and tribulations. In your hands, we find fortune and glory. Lead us on, oh Dungeon Master.


## 主角和地下城主的系统消息

In [ ]:
protagonist_system_message = SystemMessage(
    content=(
        f"""{game_description}
永远不要忘记你是主角 {protagonist_name}，而我是讲述者 {storyteller_name}。
你的角色描述如下：{protagonist_description}。
你将提出你计划采取的行动，而我将解释当你采取这些行动时会发生什么。
从 {protagonist_name} 的第一人称视角说话。
对于描述你自己的身体动作，用'*'包裹你的描述。
不要改变角色！
不要从 {storyteller_name} 的视角说话。
不要忘记在说完话时说'轮到你了，{storyteller_name}。'
不要添加任何其他内容。
记住你是主角 {protagonist_name}。
从你的视角说完话就立即停止。
"""
    )
)

storyteller_system_message = SystemMessage(
    content=(
        f"""{game_description}
永远不要忘记你是讲述者 {storyteller_name}，而我是主角 {protagonist_name}。
你的角色描述如下：{storyteller_description}。
我将提出我计划采取的行动，而你将解释当我采取这些行动时会发生什么。
从 {storyteller_name} 的第一人称视角说话。
对于描述你自己的身体动作，用'*'包裹你的描述。
不要改变角色！
不要从 {protagonist_name} 的视角说话。
不要忘记在说完话时说'轮到你了，{protagonist_name}。'
不要添加任何其他内容。
记住你是讲述者 {storyteller_name}。
从你的视角说完话就立即停止。
"""
    )
)

## 使用 LLM 创建详细的任务描述

In [ ]:
quest_specifier_prompt = [
    SystemMessage(content="你可以使任务更具体。"),
    HumanMessage(
        content=f"""{game_description}
        
        你是讲述者 {storyteller_name}。
        请让任务更具体。要有创意和想象力。
        请用不超过 {word_limit} 个字回复详细的任务。
        直接对主角 {protagonist_name} 说话。
        不要添加任何其他内容。"""
    ),
]
specified_quest = ChatOpenAI(temperature=1.0)(quest_specifier_prompt).content

print(f"原始任务：\n{quest}\n")
print(f"详细任务：\n{specified_quest}\n")

Original quest:
Find all of Lord Voldemort's seven horcruxes.

Detailed quest:
Harry, you must venture to the depths of the Forbidden Forest where you will find a hidden labyrinth. Within it, lies one of Voldemort's horcruxes, the locket. But beware, the labyrinth is heavily guarded by dark creatures and spells, and time is running out. Can you find the locket before it's too late?



## 主循环

In [16]:
protagonist = DialogueAgent(
    name=protagonist_name,
    system_message=protagonist_system_message,
    model=ChatOpenAI(temperature=0.2),
)
storyteller = DialogueAgent(
    name=storyteller_name,
    system_message=storyteller_system_message,
    model=ChatOpenAI(temperature=0.2),
)

In [17]:
def select_next_speaker(step: int, agents: List[DialogueAgent]) -> int:
    idx = step % len(agents)
    return idx

In [18]:
max_iters = 6
n = 0

simulator = DialogueSimulator(
    agents=[storyteller, protagonist], selection_function=select_next_speaker
)
simulator.reset()
simulator.inject(storyteller_name, specified_quest)
print(f"({storyteller_name}): {specified_quest}")
print("\n")

while n < max_iters:
    name, message = simulator.step()
    print(f"({name}): {message}")
    print("\n")
    n += 1

(Dungeon Master): Harry, you must venture to the depths of the Forbidden Forest where you will find a hidden labyrinth. Within it, lies one of Voldemort's horcruxes, the locket. But beware, the labyrinth is heavily guarded by dark creatures and spells, and time is running out. Can you find the locket before it's too late?


(Harry Potter): I take a deep breath and ready my wand. I know this won't be easy, but I'm determined to find that locket and destroy it. I start making my way towards the Forbidden Forest, keeping an eye out for any signs of danger. As I enter the forest, I cast a protective spell around myself and begin to navigate through the trees. I keep my wand at the ready, prepared for any surprises that may come my way. It's going to be a long and difficult journey, but I won't give up until I find that horcrux. It is your turn, Dungeon Master.


(Dungeon Master): As you make your way through the Forbidden Forest, you hear the rustling of leaves and the snapping of twigs. S